In [ ]:
import sqlite3
import hashlib
import requests
import json
import pandas as pd
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path

def css_selectors():
    from bs4 import BeautifulSoup
    html = '<div class="foo" data-value="10"></div><div class="foo" data-value="20"></div>'
    soup = BeautifulSoup(html, 'html.parser')
    total = sum(int(div['data-value']) for div in soup.find_all('div', class_='foo'))
    print("Sum of data-value attributes:", total)

def github_task():
    data = {"email": "22f3001220@ds.study.iitm.ac.in"}
    with open("email.json", "w") as f:
        json.dump(data, f)
    print("Created email.json. Push to GitHub manually.")

def sqlite_task():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE tickets (type TEXT, units INTEGER, price REAL)
    """)
    cursor.executemany("INSERT INTO tickets VALUES (?, ?, ?)", [
        ("GOLD", 25, 0.8), ("Bronze", 837, 1.43), ("BRONZE", 714, 0.66),
        ("Gold", 158, 1.31), ("SILVER", 425, 1.41)
    ])
    cursor.execute("""
        SELECT SUM(units * price) FROM tickets WHERE UPPER(type) = 'GOLD'
    """)
    print("Total sales for Gold tickets:", cursor.fetchone()[0])
    conn.close()

def markdown_task():
    markdown_content = """
# Step Tracking Report

## Methodology
- **Important**: Data is collected from a fitness tracker.
- *Note*: Steps are averaged over the week.

## Daily Steps Data
| Day   | Steps |
|-------|-------|
| Monday | 8000 |
| Tuesday | 7500 |

## Code Example
```python
print("Analyzing steps data")
```
"""
    with open("report.md", "w") as f:
        f.write(markdown_content)
    print("Markdown file 'report.md' created.")

def colab_task():
    email = "22f3001220@ds.study.iitm.ac.in"
    hashed = hashlib.sha256(f"{email} 2025".encode()).hexdigest()[-5:]
    print("Colab verification hash:", hashed)

def fastapi_task():
    app = FastAPI()
    app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["GET"])
    df = pd.read_csv("students.csv")

    @app.get("/api")
    async def get_students():
        return df.to_dict(orient="records")

    print("Run FastAPI with: uvicorn filename:app --reload")

def main():
    tasks = {
        1: css_selectors,
        2: github_task,
        3: sqlite_task,
        4: markdown_task,
        5: colab_task,
        6: fastapi_task,
    }

    print("Select a problem to solve:")
    print("1: CSS Selectors")
    print("2: GitHub Task")
    print("3: SQLite Query")
    print("4: Markdown Documentation")
    print("5: Google Colab Verification")
    print("6: FastAPI Student Data Server")

    choice = int(input("Enter choice: "))
    if choice in tasks:
        tasks[choice]()
    else:
        print("Invalid choice.")

if __name__ == "__main__":
    main()


In [ ]:
import openai

def get_llm_response(prompt):
    """Function to interact with OpenAI's API and get the response."""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an expert tutor for IIT Madras' Data Science course."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

# List of problems provided throughout the project
problems = [
    "Explain the concept of gravitational lensing to a high school student who understands basic physics, including how it’s observed and its significance in astronomy.",
    "Explain how to tie a bow tie using a step-by-step approach similar to tying a shoelace or necktie.",
    "Given a customer service transcript, analyze if the customer is satisfied by evaluating if their concerns were addressed.",
    "You are a seasoned software engineer. Explain how to fix a software bug in legacy code, including the debugging and testing process.",
    "Provide a summary of the concept of machine learning and then give a simple example, like an email spam filter.",
    "Organize a list of items in JSON format, each containing a name and description.",
    "Is this method likely to work as intended? First, explain your reasoning step by step, then give a Yes or No answer.",
]

def main():
    """Main function to handle user interaction."""
    print("Select a problem to solve:")
    for idx, problem in enumerate(problems, start=1):
        print(f"{idx}. {problem[:50]}...")

    choice = int(input("Enter the number of the problem you want to solve: "))
    if 1 <= choice <= len(problems):
        print("\nSolving the problem...")
        answer = get_llm_response(problems[choice - 1])
        print("\nAnswer:\n", answer)
    else:
        print("Invalid choice. Please select a valid number.")

if __name__ == "__main__":
    main()


In [2]:
# Install required libraries
!pip install fastapi uvicorn nest-asyncio pyngrok

# Import necessary modules
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from threading import Thread
import uvicorn

# Allow FastAPI to run inside Colab
nest_asyncio.apply()

# Authenticate ngrok (Your provided token)
!ngrok config add-authtoken 2uypxZdRXmGsTHXb98Yb2TdvaTu_693yLdKNantzwu8T39qo4

# Create FastAPI app
app = FastAPI()

# Define the available problems and their solutions
problems = {
    "Problem 1": "Solution to Problem 1",
    "Problem 2": "Solution to Problem 2",
    "Problem 3": "Solution to Problem 3",
    # Add all your collected problems here
}

# Define request model
class ProblemRequest(BaseModel):
    problem_name: str

# Endpoint to list all problems
@app.get("/problems")
def list_problems():
    return {"available_problems": list(problems.keys())}

# Endpoint to solve a selected problem
@app.post("/solve")
def solve_problem(request: ProblemRequest):
    solution = problems.get(request.problem_name, "Problem not found.")
    return {"problem": request.problem_name, "solution": solution}

# Expose API using ngrok
public_url = ngrok.connect(8000).public_url
print(f"Public API URL: {public_url}")

# Function to start FastAPI server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run API server in the background
thread = Thread(target=run)
thread.start()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public API URL: https://e04e-34-168-202-199.ngrok-free.app
